In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import importlib
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from itertools import product
from collections import defaultdict

In [2]:
# 获取项目根目录
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

# 导入数据
processed_path = Path(project_root) / 'data' / 'processed'
learning_data = pd.read_csv(processed_path / 'Task2_processed.csv')

In [94]:
import src.Bayesian_new.problems.model as model
importlib.reload(model)
from src.Bayesian_new.problems.model import SingleRationalModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_base

#### Fig 1C

In [26]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig1_Ntrial

# 初始化处理器
plot_fig1_ntrial = Fig1_Ntrial()

In [43]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig1_ntrial.plot_trial_number(learning_data,
                                figsize=(4, 5),
                                color_1='#A6A6A6',
                                color_2='#A6A6A6',
                                color_3='#A6A6A6',
                                label_1='Exp 1',
                                label_2='Exp 2',
                                label_3='Exp 3',
                                save_path=plots_dir / 'Fig1C_ntrial.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig1C_ntrial.png


#### Fig 1D

In [124]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig1_Acc

# 初始化处理器
plot_fig1_acc = Fig1_Acc()

In [125]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig1_acc.plot_accuracy(learning_data,
                    block_size= 64,
                    figsize=(15, 5),
                    widths=(0.46,1.6,0.83),
                    save_path=plots_dir / 'Fig1D_accuracy.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig1D_accuracy.png


#### Fig 1E

In [44]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig1_Oral

# 初始化处理器
plot_fig1_oral = Fig1_Oral()

In [116]:
# 导入被试数据
iSub = 7
subject_data = learning_data[learning_data['iSub'] == iSub]
subject_data = subject_data.reset_index(drop=True)
ncats=2

In [ ]:
# 绘制单帧图片
plots_dir = Path(project_root) / 'results' / 'Nips_figures' 
plot_fig1_oral.plot_human_trajactory(ncats, subject_data, 1, plots_dir, 125)

完成：仅绘制行 [125]，图表已保存至 /home/yangjiong/CategoryLearning/results/Nips_figures/choice*/ 文件夹。


In [ ]:
plot_fig1_oral.plot_human_trajactory(ncats, subject_data, 2, plots_dir, 125)

完成：仅绘制行 [125]，图表已保存至 /home/yangjiong/CategoryLearning/results/Nips_figures/choice*/ 文件夹。


In [ ]:
# 导入模型数据
result_path = Path(project_root) / 'results' / 'Bayesian_new'
fitting_results = joblib.load(result_path / 'M_Base_fitting_results.joblib')

In [117]:
sub_result = fitting_results[iSub]
step_results = sub_result['step_results']
cond = sub_result['condition']

model  = SingleRationalModel(config_base, condition=cond)
model_centers = plot_fig1_oral.get_model_centers(step_results,
                                                 model,
                                                 ncats=ncats)
model_centers['iSub']     = iSub
model_centers['iTrial']   = model_centers.index + 1

In [120]:
# 绘制单帧图片
plots_dir = Path(project_root) / 'results' / 'Nips_figures' 
plot_fig1_oral.plot_model_trajactory(ncats, model_centers, 1, plots_dir, 10)

完成模型轨迹绘制：行 [10]，图表已保存至 /home/yangjiong/CategoryLearning/results/Nips_figures/choice*/ 文件夹。


In [123]:
plot_fig1_oral.plot_model_trajactory(ncats, model_centers, 2, plots_dir, 29)

完成模型轨迹绘制：行 [29]，图表已保存至 /home/yangjiong/CategoryLearning/results/Nips_figures/choice*/ 文件夹。


#### Fig 3B

In [35]:
# 加载拟合结果
# base_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_new' / 'M_Base_fitting_results.joblib')
# forget_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_new' / 'M_Fgt_fitting_results_400.joblib')
cluster_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_cl_max7_randp_k1_acc7.joblib')
# fgtcluster_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_fgt_cl_rand7_randp_k1_acc7.joblib')

In [ ]:
def get_prediction(fitting_results, learning_data):
    predict_results = {}

    for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
        condition = subject_data['condition'].iloc[0]
        model_base = SingleRationalModel(config_base, condition=condition)
        s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
                subject_data["choice"].values,
                subject_data["feedback"].values, 
                subject_data["category"].values)
        
        step_results = fitting_results[iSub].get('step_results', fitting_results[iSub].get('best_step_results'))
        results = model_base.predict_choice(s_data, step_results, use_cached_dist=False, window_size=16)

        predict_results[iSub] = {
            'condition': condition,
            'true_acc': results['true_acc'],
            'pred_acc': results['pred_acc'],
            'sliding_true_acc': results['sliding_true_acc'],
            'sliding_pred_acc': results['sliding_pred_acc'],
            'sliding_pred_acc_std': results['sliding_pred_acc_std']
        }
    return predict_results

In [37]:
# base_predict_results = get_prediction(base_results, learning_data)
# forget_predict_results = get_prediction(forget_results, learning_data)
cluster_predict_results = get_prediction(cluster_results, learning_data)
# fgtcluster_predict_results = get_prediction(fgtcluster_results, learning_data)

In [38]:
# joblib.dump(base_predict_results, Path(project_root) / 'results' / 'Bayesian_recon' / 'M_base_prediction.joblib')
# joblib.dump(forget_predict_results, Path(project_root) / 'results' / 'Bayesian_recon' / 'M_fgt_prediction.joblib')
joblib.dump(cluster_predict_results, Path(project_root) / 'results' / 'Bayesian_recon' / 'M_cluster_prediction.joblib')
# joblib.dump(fgtcluster_predict_results, Path(project_root) / 'results' / 'Bayesian_recon' / 'M_fgt_cluster_prediction.joblib')

['/home/yangjiong/CategoryLearning/results/Bayesian_recon/M_cluster_prediction.joblib']

In [ ]:
# 加载拟合结果
base_predict_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_base_prediction.joblib')
forget_predict_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_fgt_prediction.joblib')
cluster_predict_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_cluster_prediction.joblib')
fgtcluster_predict_results = joblib.load(Path(project_root) / 'results' / 'Bayesian_recon' / 'M_fgt_cluster_prediction.joblib')

In [47]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig3

# 初始化处理器
plot_fig3 = Fig3()

In [50]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3.plot_error_comparison(results_1=base_predict_results,
                              results_2=cluster_predict_results,
                              results_3=forget_predict_results,
                              results_4=fgtcluster_predict_results,
                              mc_method='fdr_bh',
                              figsize=(6, 5),
                              color_1='#DDAA33',
                              color_2='#478ECC',
                              color_3='#45B53F',
                              color_4='#F39972',
                            #   label_1='Base',
                            #   label_2='Fgt',
                              save_path=plots_dir / 'Fig3B_error.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3B_error.png
Raw p-values:         [0.     0.     0.0219]
Corrected p-values (fdr_bh): [0.     0.     0.0219]
Reject null hypotheses: [ True  True  True]


#### Fig 3C

In [39]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig3

# 初始化处理器
plot_fig3 = Fig3()

In [117]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3.plot_acc_comparison(results_1=base_predict_results,
                              results_2=forget_predict_results,
                              subject_id=14,
                              figsize=(8, 5),
                              color_1='#DDAA33',
                              color_2='#45B53F',
                              color_true='#A6A6A6',
                              label_1='Base',
                              label_2='Fgt',
                              save_path=plots_dir / 'Fig3C_base_fgt.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3C_base_fgt.png


In [ ]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3.plot_acc_comparison(results_1=base_predict_results,
                              results_2=fgtcluster_predict_results,
                              subject_id=14,
                              figsize=(8, 5),
                              color_1='#DDAA33',
                              color_2='#F39972',
                              color_true='#A6A6A6',
                              label_1='Base',
                              label_2='Fgt+Jump',
                              save_path=plots_dir / 'Fig3C_base_fgt_cluster.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3C_base_fgt_cluster.png


In [40]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3.plot_acc_comparison(results_1=base_predict_results,
                              results_2=cluster_predict_results,
                              subject_id=14,
                              figsize=(8, 5),
                              color_1='#DDAA33',
                              color_2='#478ECC',
                              color_true='#A6A6A6',
                              label_1='Base',
                              label_2='Jump',
                              save_path=plots_dir / 'Fig3C_base_cluster.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3C_base_cluster.png


#### Fig 3D

In [41]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig3D

# 初始化处理器
plot_fig3d = Fig3D()

In [10]:
oral_hypos_list = {}

for _, subj_df in learning_data.groupby('iSub'):
    iSub   = int(subj_df['iSub'].iloc[0])
    cond   = int(subj_df['condition'].iloc[0])
    model  = SingleRationalModel(config_base, condition=cond)

    centres = subj_df[['feature1_oral','feature2_oral',
                       'feature3_oral','feature4_oral']].values
    choices = subj_df['choice'].values

    oral_hypos_list[iSub] = plot_fig3d.get_oral_hypos_list(
        (centres, choices), model)

In [12]:
oral_hypo_hits = {}

for iSub, hypos in oral_hypos_list.items():
    condition = learning_data[learning_data['iSub'] ==
                                iSub]['condition'].iloc[0]
    target_value = 0 if condition == 1 else 42

    hits = []  # 用于存储每个 trial 的 hit 值
    for trial_hypos in hypos:
        hit = 1 if target_value in trial_hypos else 0
        hits.append(hit)  # 将 hit 值添加到列表中

    # 计算滑动平均
    window_size = 16
    rolling_hits = pd.Series(hits).rolling(
        window=window_size, min_periods=window_size).mean().tolist()

    oral_hypo_hits[iSub] = {
        'iSub': iSub,
        'condition': condition,
        'rolling_hits': rolling_hits
    }

In [15]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3d.plot_k_comparison(oral_hypo_hits,
                             results_1=base_results,
                             results_2=forget_results,
                             subject_id=14,
                             figsize=(8, 5),
                             color_1='#DDAA33',
                             color_2='#45B53F',
                             color_true='#A6A6A6',
                             label_1='Base',
                             label_2='Fgt',
                             save_path=plots_dir / 'Fig3D_base_fgt.png')

[Fig3D] saved → /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3D_base_fgt.png


In [16]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3d.plot_k_comparison(oral_hypo_hits,
                             results_1=base_results,
                             results_2=fgtcluster_results,
                             subject_id=14,
                             figsize=(8, 5),
                             color_1='#DDAA33',
                             color_2='#F39972',
                             color_true='#A6A6A6',
                             label_1='Base',
                             label_2='Fgt+Jump',
                             save_path=plots_dir / 'Fig3D_base_fgt_cluster.png')

[Fig3D] saved → /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3D_base_fgt_cluster.png


In [42]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig3d.plot_k_comparison(oral_hypo_hits,
                             results_1=base_results,
                             results_2=cluster_results,
                             subject_id=14,
                             figsize=(8, 5),
                             color_1='#DDAA33',
                             color_2='#478ECC',
                             color_true='#A6A6A6',
                             label_1='Base',
                             label_2='Jump',
                             save_path=plots_dir / 'Fig3D_base_cluster.png')

[Fig3D] saved → /home/yangjiong/CategoryLearning/results/Nips_figures/Fig3D_base_cluster.png


#### Fig 4A

In [56]:
import src.nips_plot as nips_plot
importlib.reload(nips_plot)
from src.nips_plot import Fig4A

# 初始化处理器
plot_fig4a = Fig4A()

In [58]:
plots_dir = Path(project_root) / 'results' / 'Nips_figures'
plot_fig4a.plot_amount(results=fgtcluster_results,
                             subject_id=14,
                             figsize=(8, 5),
                             color_1='#F39972',
                             color_2='#A4A0B7',
                             label_1='Exploitation',
                             label_2='Exploration',
                             save_path=plots_dir / 'Fig4A_amount.png')

Figure saved to /home/yangjiong/CategoryLearning/results/Nips_figures/Fig4A_amount.png
